In [18]:
import numpy as np

from gabor_filters._my_custom_utils import solve_filter_parameters


In [14]:
def create_a_set_of_gabo_filters(fmax=0.327, k=np.sqrt(2), p=0.5, u=6, v=8, row=43, col=43):
#     row=43, col=43 # size of image
#     fmax = 0.327 # maximum frequency

#     k = np.sqrt(2) #frequency ratio or factor for selecting filter frequencies

#     p = 0.5 # crossing point between two consecutive filters, default 0.5
#     # pf = np.sqrt(0.99) #energy to include in the filters

#     u = 6 #number of frequencies
#     v = 8 #number of orientation

    gamma, eta = solve_filter_parameters(k, p, u, v) # smoothing parameters

    GaborFilterBank = [None]*u*v 
      
    for i in range(0,u):
        fu = fmax/k**i # frequency of the filter
        for j in range(0,v):
            theta = j/v*np.pi; # orientation of the filter 

            GaborFilterIJ = GaborFilter()
            GaborFilterIJ.frequency = fu
            GaborFilterIJ.orientation = theta
            GaborFilterIJ.compute_a_gabor_filter(gamma, eta, row, col) 

            # GaborFilterIJ.showParameters()

            GaborFilterBank[i*v+j] = GaborFilterIJ
    
    return GaborFilterBank

NameError: name 'np' is not defined